In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
train_path = '/content/drive/MyDrive/Aiffel_Hackathon/Data/Training_Data_Clean.csv'
test_path = '/content/drive/MyDrive/Aiffel_Hackathon/Data/Test_Data_Clean.csv'

train_csv = pd.read_csv(train_path)
train_csv.head()

,Unnamed: 0,Original_Filename_x,Gold_Filename,Document,Full_Text,Summary_Text
0,0,15786_819157_2,15786_819156_1,15786,6 chief executive officer’s review the berong...,3 chairman’s statement in the 18 months since...
1,1,15783_819108_2,15783_819107_1,15783,chief executive officer’s statement the beron...,chairman s statement it has always been your ...
2,2,15782_819091_2,15782_819090_1,15782,chief executive officer’s statement the compa...,chairman’s statement the celestial ipilan ore...
3,3,15787_819176_2,15787_819175_1,15787,page 6 toledo mining corporation plc annual r...,page 5 i should like to begin my message with...
4,4,15816_820156_2,15816_820155_1,15816,3 chief executive’s statement tomkins plc rep...,2 tomkins plc report accounts 2003 chairman’s...


In [ ]:
train = train_csv[['Document','Full_Text', 'Summary_Text']]
train.head(2)

,Document,Full_Text,Summary_Text
0,15786,6 chief executive officer’s review the berong...,3 chairman’s statement in the 18 months since...
1,15783,chief executive officer’s statement the beron...,chairman s statement it has always been your ...


In [ ]:
train.to_csv('/content/drive/MyDrive/Aiffel_Hackathon/Data/Training_Data_Clean_Short.csv', index=False)

In [ ]:
train_path = '/content/drive/MyDrive/Aiffel_Hackathon/Data/Training_Data_Clean_Short.csv'
train_data = pd.read_csv(train_path)
train_data.head(2)

,Document,Full_Text,Summary_Text
0,15786,6 chief executive officer’s review the berong...,3 chairman’s statement in the 18 months since...
1,15783,chief executive officer’s statement the beron...,chairman s statement it has always been your ...


In [ ]:
%cd /content/drive/MyDrive/Aiffel_Hackathon/Data

/content/drive/MyDrive/Aiffel_Hackathon/Data


In [ ]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
import tensorflow as tf

In [ ]:
tfrecords_filename = 'train.tfrecords'

writer = tf.io.TFRecordWriter(tfrecords_filename)

original_tables = []

for item in train_data.itertuples():
    example = tf.train.Example(features=tf.train.Features(feature={
        'Document': _int64_feature(item.Document),
        'Full_Text': _bytes_feature(str.encode(item.Full_Text)),
        'Summary_Text': _bytes_feature(str.encode(item.Summary_Text)),
        }))
    
    writer.write(example.SerializeToString())

writer.close()

In [ ]:
raw_dataset = tf.data.TFRecordDataset(tfrecords_filename)

for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

features {
  feature {
    key: "Document"
    value {
      int64_list {
        value: 15786
      }
    }
  }
  feature {
    key: "Full_Text"
    value {
      bytes_list {
        value: " 6 chief executive officer\342\200\231s review the berong nickel mine , in which toledo has a 56.1 percent economic interest , successfully recommenced operations in may 2011.this became possible due to favourable market conditions for high grade saprolite nickel ores and strong nickel prices which have stayed above usdollar 20,000 tonne since august 2010. prior to reopening the berong mine , toledo developed a new strategy with its jv partners in berong nickel corporation bnc to target production of nickel ore containing 1.8 percent nickel , which is currently being implemented . the 1.8 percent plan was developed consistent with growing market demands for this ore type . the recent significant changes in market demand for high grade saprolite nickel ore have opened up sound prospects for our de

In [ ]:
print(train_data['Full_Text'][0])

 6 chief executive officer’s review the berong nickel mine , in which toledo has a 56.1 percent economic interest , successfully recommenced operations in may 2011.this became possible due to favourable market conditions for high grade saprolite nickel ores and strong nickel prices which have stayed above usdollar 20,000 tonne since august 2010. prior to reopening the berong mine , toledo developed a new strategy with its jv partners in berong nickel corporation bnc to target production of nickel ore containing 1.8 percent nickel , which is currently being implemented . the 1.8 percent plan was developed consistent with growing market demands for this ore type . the recent significant changes in market demand for high grade saprolite nickel ore have opened up sound prospects for our deposits . historically , the development of exports of high grade saprolite nickel ore began from new caledonia during the 1950 s by japanese ferronickel producers . later , they extended their ore source 

In [ ]:
print(train_data['Summary_Text'][0])

 3 chairman’s statement in the 18 months since the beginning of toledo’s 2011 financial year , our management has worked tirelessly to extract greater value from our nickel investments located in the philippines . it was very pleasing therefore to be able to announce in april that sales contracts had been secured to finance the reopening of the berong mine , which duly occurred in may . the intention is now to produce 450,000 wet metric tonnes wmt of nickel ore this calendar year rising to 750,000 wmt annually thereafter . the expected net revenue generated from ore sales will be more than sufficient at current nickel prices for the joint owners of berong to fund an exploration programme of the area around berong and undertake engineering studies to improve ship loading capability and determine the optimal technology for local processing of ore . at the ipilan project , on the eastern and opposite side of palawan island to berong , significant progress has been made towards being able 

현재 Bigbird는 SOTA 논문이기 때문에 구현된 코드가 많이 없다. 토크나이저도 제대로 불러와지지 않아서 AutoTokenizer를 이용했지만 제대로 되지 않았다. 따라서 google research 깃허브에서 구현한 코드를 차근차근 따라가며 모델을 구현할 계획이다.  
해당 코드는 Tensorflow 기반이며 TPU를 사용한다. 따라서 연산이 원활하게 이루어질 수 있도록 기존의 dataset을 tensorflow의 것으로 바꿔 주어야 한다. pandas에서 지원하는 dataframe만 이용하다가 직관적이지 않은 dataset을 사용하니 모든게 생소하게 다가왔다. 하지만 이번에는 모델의 구조를 직접 볼 수 있는 만큼 좋은 결과가 있지 않을까 생각한다.